In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

In [2]:
# Загрузка и предварительная обработка данных
path = "DataSets/dataframe_benign_brute_xss_sql.csv"
df = pd.read_csv(path)
df.columns = [col.lstrip() for col in df.columns]

label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])

X = df.drop('Label', axis=1)
y = df['Label']

In [3]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

In [4]:
# Применение SMOTE для оверсэмплинга
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [5]:
# Настройка гиперпараметров с помощью GridSearchCV
param_grid = {
    'n_estimators': [100, 200],  # количество деревьев
    'max_depth': [10, 20],       # максимальная глубина дерева
    # Можно добавить другие параметры для настройки
}

clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(clf, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_resampled, y_resampled)

# Лучшая модель из GridSearch
best_model = grid_search.best_estimator_

In [6]:
# Предсказание и оценка
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Лучшая точность с GridSearch:", accuracy)
print("Отчет по классификации:\n", classification_report(y_test, y_pred))

# print(y.value_counts())
# print(y_resampled.value_counts())

Лучшая точность с GridSearch: 0.9943354425068395
Отчет по классификации:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    117633
           1       0.76      0.57      0.65      1061
           2       1.00      0.69      0.82        13
           3       0.36      0.58      0.45       455

    accuracy                           0.99    119162
   macro avg       0.78      0.71      0.73    119162
weighted avg       1.00      0.99      0.99    119162

